# Desk Posture Nudge 

**Problem** 

We’ve all been there: rubbing our backs and wondering if it’s an injury or just too many minutes of bad posture. We’d rather prevent than fix. So we built a tiny on-device vision model that checks whether your posture is good enough and nudges you only when it really matters. If and only if you stay in bad posture for **N** seconds, it sends a gentle alert. Sit up straight, and it quietly resets.

<p align="center">
  <img src="sitting-posture.webp" alt="Sitting posture" width="50%">
</p>

**Data:**
- Source: [dataset link1](https://universe.roboflow.com/dataset-sqm0h/sitting-posture-ezkda), [link2](https://universe.roboflow.com/min-bnvzi/sitting-posture-v2)
- Split: **70/30** (train/val), balanced datasets. 

**Dataset prep:**
- **Auto-Orient:** applied so all images share the same upright orientation.

- **Resize:** **Fit within 96×96** (no stretch)  

- **Remap Classes:** 
  Merged granular variants into a single label.
  - `backwardbadposture`, `forwardbadposture` → **badposture**
  - `goodposture` 


**Model:**

[FocoosAI](https://focoos.ai/) offers many pretrained classification models in different sizes, we tested: 

- fai-cls-n-coco  (nano, optimized for Arduino Nicla Vision) 
- fai-cls-s-coco  (small)
- fai-cls-m-coco  (medium).

In [ ]:
# setup the environment
# !uv venv --python=3.12
# !source .venv/bin/activate
!uv pip install 'focoos[onnx-cpu] @ git+https://github.com/FocoosAI/focoos@feat/train-on-AppleSilicon';

In [ ]:
import os
from focoos.infer.quantizer import OnnxQuantizer, QuantizationCfg
from focoos import ModelManager, RuntimeType, InferModel
from dotenv import load_dotenv

load_dotenv()
IMAGE_SIZE = 96

**Training recipe (safe defaults).**
- Backbone: `fai-cls-n-coco`
- Batch size: **16** (or 32 if available)
- Optimizer: **AdamW**, weight decay **1e-4**
- Square: 1.0

In [ ]:
!bash -c 'focoos train --dataset sitting_posture_dataset.zip \
    --datasets-dir . \
    --dataset-layout cls_folder \
    --model fai-cls-s-coco \
    --im-size {IMAGE_SIZE} \
    --device mps'

We exports **ONNX CPU** with fixed input size (`IMAGE_SIZE`), **static shape** (`dynamic_axes=False`), graph **simplification** (`simplify_onnx=True`), **opset 18**, to `export/my_hub_model/`.

In [ ]:
# hub = FocoosHUB(api_key=FOCOOS_API_KEY)
model = ModelManager.get("fai-cls-m-coco-22d2fb83-7008-4475-9669-b48cedc76f8a")

exported_model = model.export(
    runtime_type=RuntimeType.ONNX_CPU,
    image_size=IMAGE_SIZE,
    dynamic_axes=False,
    simplify_onnx=True,  # simplify and optimize onnx model graph
    onnx_opset=18,
    out_dir=os.path.join("export/", "my_hub_model"),
)

**Calibration images**

We sampled the calibration set **from the validation split**, selecting ~30–40 images with a **good diversity** (both classes, different subjects, lighting, viewpoints, scales).

In [ ]:
quantization_cfg = QuantizationCfg(
    size=IMAGE_SIZE,
    calibration_images_folder=str(
        "calibration_images",
    ),
    format="QO",
    per_channel=False,  # Per-channel quantization: each channel has its own scale/zero-point → more accurate,
    # especially for convolutions, at the cost of extra memory and computation.
    normalize_images=True,
)

quantizer = OnnxQuantizer(
    input_model_path=exported_model.model_path, cfg=quantization_cfg
)
model_path = quantizer.quantize(
    benchmark=True  # benchmark bot fp32 and int8 models
)
quantized_model = InferModel(model_path, runtime_type=RuntimeType.ONNX_CPU)

In [ ]:
path_val = "sitting_posture_dataset/valid/goodposture"
supported = (".jpg", ".jpeg", ".png", ".webp", ".gif")

good = bad = total = 0

for name in sorted(os.listdir(path_val)):
    if not name.lower().endswith(supported):
        continue
    full_path = os.path.join(path_val, name)
    out = quantized_model.infer(image=full_path)

    for det in getattr(out, "detections", []):
        total += 1
        if det.label == "badposture":
            bad += 1
        else:
            good += 1

print(f"total={total}  bad={bad}  good={good}")

**Possible deployment ideas (on device).**
- **Confidence gate** (e.g., `p_max < 0.6` -> ignore frame).
- **Timer N** seconds before triggering the nudge (LED/Buzzer/Serial).
- Reset timer when posture returns to **Good**.